Sitas notebookas paima alignintus sgenus, sgenuose yra bruksneliu -, kuriuos reikia panaikinti
1. atmetam tuos kurie neprasideda ATG pradzios kodonu
2. minusiukus (outlier'ius) šaliname 2 būdais: 
  1. jei pozicijoje dominuoja minusiukai - šaliname sekas kurios maišo, dedam i blacklist
  2. jei dominuoja ne minusiukai - užpildome koncensusu ir transliuojam nukleotida i proteina, kad patikrintume ar istacius raide pagal konsensusa neatsirado pabaigos kodonas viduryje geno
3. rezultate gauna "ncbi_sgene_good_unique_aligned_cleaned.csv" kuriame yra sgenai be bruksneliu


In [1]:
import pandas as pd
from pathlib import Path

In [2]:
blacklist = [
    "1_07011ff250ef4a4ace2ef9b43d5c2ba1", # pradzios kodonas ne ATG
    "1_bc8364434f1c405884ffb19232630d57", # pradzios kodonas ne ATG
    "1_4cfad326ce4e88375a4940a9d520f1dd", # pradzios kodonas ne ATG
    
    "1_25080f7a80094f94da370709b0d34695", # minusiukai nuo 822 iki 830
    
    "6_21591cfd60c885e07d7f57962f053be0", # minusiukai nuo 672 iki 680
    "1_370e85eb954793235737752b3be9ff8d", # minusiukai nuo 672 iki 680
    
    "1_73ab13457a0970f920022ef0d2e15058", # minusiukai nuo 797 iki 799
    
    "1_5770e81e98ccef6a23b405da967dfb8d", # minusiukai nuo 0 iki 26
    "1_ca6f6b227c6eef0951de1e28652b7e2d", # minusiukai nuo 0 iki 26
    "1_83783011e3066d4938cced16296c2604", # minusiukai nuo 0 iki 26
    "5_0e5599a7c4545edf989e6bb8ded0bcc8", # minusiukai nuo 0 iki 26
    
    "2_059f461871887e99ab81c93d6be95c63", # minusiukai nuo 34 iki 36
    "1_1c35868a205decc006af1af9332de520", # minusiukai nuo 34 iki 36
    "1_1dbcc614d9587e62d666ced83f7d67c1"  # minusiukai nuo 34 iki 36
]

def read_fasta(filename):
    sequences = []
    path = Path(filename)
    with open(path, "r") as fasta_file:
        sequence = {}
        accession = None
        for line in fasta_file:
            if '>' in line:
                if sequence and accession:
                    
                    if accession in blacklist:
                        print(f"Skipping {accession}")
                    else:
                        sequences.append({
                            "accession": accession,
                            "sgene_nucleotide": "".join(sequence)
                        })
                    
                parts = line.split(" ")
                accession = parts[0][1:].strip()
                sequence = []
            else:
                sequence.append(line.strip())
    
    df = pd.DataFrame.from_dict(sequences)
#     df.to_csv(path.with_suffix(".csv"), index=False)
    return df

df = read_fasta("good_unique.aln")


Skipping 2_059f461871887e99ab81c93d6be95c63
Skipping 1_25080f7a80094f94da370709b0d34695
Skipping 1_07011ff250ef4a4ace2ef9b43d5c2ba1
Skipping 6_21591cfd60c885e07d7f57962f053be0
Skipping 1_1c35868a205decc006af1af9332de520
Skipping 1_bc8364434f1c405884ffb19232630d57
Skipping 1_4cfad326ce4e88375a4940a9d520f1dd
Skipping 1_5770e81e98ccef6a23b405da967dfb8d
Skipping 1_1dbcc614d9587e62d666ced83f7d67c1
Skipping 1_ca6f6b227c6eef0951de1e28652b7e2d
Skipping 1_83783011e3066d4938cced16296c2604
Skipping 5_0e5599a7c4545edf989e6bb8ded0bcc8
Skipping 1_73ab13457a0970f920022ef0d2e15058
Skipping 1_370e85eb954793235737752b3be9ff8d


In [3]:
df

,accession,sgene_nucleotide
0,1_87b59e0087b7cfc624758ac0f7cc0f89,---------------------------ATGTTTG---TTTTTCTTG...
1,1_7d7d2f39b893ea3ca706781d1ca03d70,---------------------------ATGTTTG---TTTTTCTTG...
2,1_600ed6f06199b27156f02df015050fa3,---------------------------ATGTTTG---TTTTTCTTG...
3,1_543814ce04c1c7813ef42e7fe0abef00,---------------------------ATGTTTG---TTTTTCTTG...
4,1_87472661071a21fef79f2166f4e8e631,---------------------------ATGTTTG---TTTTTCTTG...
...,...,...
4364,1_5321f55a779924f878d9a69ffe18b4c1,---------------------------ATGTTTG---TTTTTCTTG...
4365,2_fe3511bb655ecf8318fa9aee6b39a4fb,---------------------------ATGTTTG---TTTTTCTTG...
4366,2_f3c41465d64388a1246062814595c060,---------------------------ATGTTTG---TTTTTCTTG...
4367,1_a7434325322e04336922df03d2033a33,---------------------------ATGTTTG---TTTTTCTTG...


In [4]:
# Verciam nucleotide seko i series, kad butu kiekvienai raidei po stulpeli
def convert(s):
    return pd.Series(list(s))

_df = df["sgene_nucleotide"].apply(convert)

In [5]:
df = pd.concat([df, _df], axis=1)

In [6]:
df

,accession,sgene_nucleotide,0,1,2,3,4,5,6,7,...,3863,3864,3865,3866,3867,3868,3869,3870,3871,3872
0,1_87b59e0087b7cfc624758ac0f7cc0f89,---------------------------ATGTTTG---TTTTTCTTG...,-,-,-,-,-,-,-,-,...,T,T,A,C,A,C,A,T,A,A
1,1_7d7d2f39b893ea3ca706781d1ca03d70,---------------------------ATGTTTG---TTTTTCTTG...,-,-,-,-,-,-,-,-,...,T,T,A,C,A,C,A,T,A,A
2,1_600ed6f06199b27156f02df015050fa3,---------------------------ATGTTTG---TTTTTCTTG...,-,-,-,-,-,-,-,-,...,T,T,A,C,A,C,A,T,A,A
3,1_543814ce04c1c7813ef42e7fe0abef00,---------------------------ATGTTTG---TTTTTCTTG...,-,-,-,-,-,-,-,-,...,T,T,A,C,A,C,A,T,A,A
4,1_87472661071a21fef79f2166f4e8e631,---------------------------ATGTTTG---TTTTTCTTG...,-,-,-,-,-,-,-,-,...,T,T,A,C,A,C,A,T,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4364,1_5321f55a779924f878d9a69ffe18b4c1,---------------------------ATGTTTG---TTTTTCTTG...,-,-,-,-,-,-,-,-,...,T,T,A,C,A,C,A,T,A,A
4365,2_fe3511bb655ecf8318fa9aee6b39a4fb,---------------------------ATGTTTG---TTTTTCTTG...,-,-,-,-,-,-,-,-,...,T,T,A,C,A,C,A,T,A,A
4366,2_f3c41465d64388a1246062814595c060,---------------------------ATGTTTG---TTTTTCTTG...,-,-,-,-,-,-,-,-,...,T,T,A,C,A,C,A,T,A,A
4367,1_a7434325322e04336922df03d2033a33,---------------------------ATGTTTG---TTTTTCTTG...,-,-,-,-,-,-,-,-,...,T,T,A,C,A,C,A,T,A,A


In [7]:
df.to_csv("frequencies_df.csv")

In [8]:
frequencies = _df.apply(pd.value_counts)

In [9]:
frequencies = frequencies.T

In [10]:
frequencies = frequencies.fillna(0)
frequencies[frequencies['-'] > 0].to_csv("frequences.csv")
frequencies

,-,A,C,G,T
0,4369.0,0.0,0.0,0.0,0.0
1,4369.0,0.0,0.0,0.0,0.0
2,4369.0,0.0,0.0,0.0,0.0
3,4369.0,0.0,0.0,0.0,0.0
4,4369.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
3868,0.0,0.0,4369.0,0.0,0.0
3869,0.0,4368.0,1.0,0.0,0.0
3870,0.0,0.0,0.0,0.0,4369.0
3871,0.0,4369.0,0.0,0.0,0.0


In [11]:
# Stulpeliu maske kuriuose yra tik minusiukai
mask_only_dash = (frequencies['-'] > 0) & (frequencies['A'] == 0)  & (frequencies['T'] == 0)  & (frequencies['G'] == 0) & (frequencies['C'] == 0)
mask_only_dash

0        True
1        True
2        True
3        True
4        True
        ...  
3868    False
3869    False
3870    False
3871    False
3872    False
Length: 3873, dtype: bool

In [12]:
# Stulpeliu vardai kuriuose yra tik minusiukai
columns_only_dash = mask_only_dash[mask_only_dash].index.values
columns_only_dash

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  34,  35,  36, 672, 673, 674, 675, 676, 677, 678, 679, 680,
       797, 798, 799, 822, 823, 824, 825, 826, 827, 828, 829, 830])

In [13]:
# Metam stulpelius kuriuose yra tik minusiukai
df.drop(columns_only_dash, axis=1, inplace=True) 

In [14]:
df.head()

,accession,sgene_nucleotide,27,28,29,30,31,32,33,37,...,3863,3864,3865,3866,3867,3868,3869,3870,3871,3872
0,1_87b59e0087b7cfc624758ac0f7cc0f89,---------------------------ATGTTTG---TTTTTCTTG...,A,T,G,T,T,T,G,T,...,T,T,A,C,A,C,A,T,A,A
1,1_7d7d2f39b893ea3ca706781d1ca03d70,---------------------------ATGTTTG---TTTTTCTTG...,A,T,G,T,T,T,G,T,...,T,T,A,C,A,C,A,T,A,A
2,1_600ed6f06199b27156f02df015050fa3,---------------------------ATGTTTG---TTTTTCTTG...,A,T,G,T,T,T,G,T,...,T,T,A,C,A,C,A,T,A,A
3,1_543814ce04c1c7813ef42e7fe0abef00,---------------------------ATGTTTG---TTTTTCTTG...,A,T,G,T,T,T,G,T,...,T,T,A,C,A,C,A,T,A,A
4,1_87472661071a21fef79f2166f4e8e631,---------------------------ATGTTTG---TTTTTCTTG...,A,T,G,T,T,T,G,T,...,T,T,A,C,A,C,A,T,A,A


In [15]:
frames = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                 
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
    'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W'
}

def translate(seq):
    
    seq = seq.upper()
    
    protein = []
    n = len(seq)
    if n % 3 != 0:
        print("nukleotidas nedalus is 3")
        return
    
    for i in range(0, n, 3):
        codon = seq[i:i + 3]
        frame = frames[codon]
        protein.append(frame)
        
    return "".join(protein)

In [16]:
# pakeiciam minusiukus i dazniausiai pasitaikancias reiksmes toje pozicijoje 
# ir patikrinam ar tas genas neturi stop kodonu viduryje

sgene_nucleotide_new = {}
rows_with_modified_sgene = []
for row_index, row in df.iterrows():
    nucleotide = []
    contains_dash = False 
    
    for column_name, cell in row.iteritems():
        if column_name in ['accession', 'sgene_nucleotide']:
            continue
            
        if cell == '-':
            contains_dash = True
            cell = frequencies.iloc[column_name].idxmax()
#             print(f"Replacing - with {new_cell} at {column_name}")
        nucleotide.append(cell)
    
    nucleotide_str = "".join(nucleotide)
    if contains_dash:
        # check codons
#         print("translating nucleotide to protein")
        rows_with_modified_sgene.append(row_index)
        protein = translate(nucleotide_str)
        
        if protein is None:
            # nucleotidas nedalus is 3
            continue
            
        stop = protein.index("_") # stop kodonas
        if stop is None:
            print("nera stop kodono")
            continue
            
        if len(protein) - stop > 5:
            print("stop kodonas ne gale")
            continue
            
    sgene_nucleotide_new[row_index] = nucleotide_str
# sgene_nucleotide_new

In [17]:
df['sgene_nucleotide_new'] = pd.Series(sgene_nucleotide_new)

In [18]:
df_new = df[['accession', 'sgene_nucleotide_new']]

In [19]:
df_new = df_new.rename(columns={"sgene_nucleotide_new": "sgene_nucleotide"})
df_new

,accession,sgene_nucleotide
0,1_87b59e0087b7cfc624758ac0f7cc0f89,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
1,1_7d7d2f39b893ea3ca706781d1ca03d70,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
2,1_600ed6f06199b27156f02df015050fa3,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
3,1_543814ce04c1c7813ef42e7fe0abef00,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
4,1_87472661071a21fef79f2166f4e8e631,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
...,...,...
4364,1_5321f55a779924f878d9a69ffe18b4c1,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
4365,2_fe3511bb655ecf8318fa9aee6b39a4fb,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
4366,2_f3c41465d64388a1246062814595c060,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
4367,1_a7434325322e04336922df03d2033a33,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...


In [20]:
df_new.to_csv("ncbi_sgene_good_unique_aligned_cleaned.csv")

In [21]:
with open("ncbi_sgene_good_unique_aligned_cleaned.fasta", "w") as fasta_file:
    for index, row in df_new.iterrows():
        fasta_file.write(f">{row['accession']}\n")
        fasta_file.write(f"{row['sgene_nucleotide']}\n\n")    